In [ ]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display
import numbers
from src.consts import *
from src.deck import get_deck_colour
from src.validation import validate_jumpstart_cube
from src.coherence import analyze_deck_theme_coherence_enhanced

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 


# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v3.csv')

In [3]:

def fetch_card_power_toughness(mtgo_id):
    """
    Fetch card data from Scryfall API using MTGO ID
    Returns power and toughness if the card is a creature, otherwise returns None, None
    """
    url = f"https://api.scryfall.com/cards/mtgo/{int(mtgo_id)}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        
        card_data = response.json()
        
        # Check if the card is a creature
        if 'Creature' in card_data.get('type_line', ''):
            power = card_data.get('power')
            toughness = card_data.get('toughness')
            return power, toughness
        else:
            return None, None
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for MTGO ID {mtgo_id}: {e}")
        return None, None
    except KeyError as e:
        print(f"Missing expected data for MTGO ID {mtgo_id}: {e}")
        return None, None

def display_validate_results(validation_results):
    """Display validation results in a readable format"""
    if validation_results['is_valid']:
        display(Markdown("### Cube is valid! 🎉"))
    else:
        display(Markdown("### Cube has issues ❗"))
    
    if validation_results['errors']:
        display(Markdown("#### Errors:"))
        for error in validation_results['errors']:
            display(Markdown(f"- {error}"))
    
    if validation_results['warnings']:
        display(Markdown("#### Warnings:"))
        for warning in validation_results['warnings']:
            display(Markdown(f"- {warning}"))
    
    if validation_results['deck_summaries']:
        display(Markdown("#### Deck Summaries:"))
        for deck_name, summary in validation_results['deck_summaries'].items():
            display(Markdown(f"**{deck_name}**: {summary}"))



In [4]:
def display_coherence_analysis_enhanced(coherence_results, top_n=5):
    """Display enhanced coherence analysis results including creature stats"""
    
    # Sort decks by overall coherence score
    sorted_decks = sorted(coherence_results.items(), 
                         key=lambda x: x[1]['overall_coherence'], 
                         reverse=True)
    
    display(Markdown("# Enhanced Deck Theme Coherence Analysis"))
    
    # Overall summary
    avg_coherence = sum(r['overall_coherence'] for r in coherence_results.values()) / len(coherence_results)
    display(Markdown(f"**Average Coherence Score: {avg_coherence:.1f}/100**"))
    
    # Top performing decks
    display(Markdown(f"## Top {top_n} Most Coherent Decks"))
    for i, (deck_name, analysis) in enumerate(sorted_decks[:top_n]):
        display(Markdown(f"### {i+1}. {deck_name}"))
        display(Markdown(f"- **Overall Score: {analysis['overall_coherence']:.1f}/100**"))
        display(Markdown(f"- **Expected Themes:** {', '.join(analysis['expected_themes'])}"))
        display(Markdown(f"- **Theme Match Score:** {analysis['theme_score']:.1f}"))
        display(Markdown(f"- **Color Coherence:** {analysis['color_coherence']:.1%}"))
        display(Markdown(f"- **Mana Curve Score:** {analysis['mana_curve_score']:.1%}"))
        
        # Creature stats
        creature_stats = analysis['creature_stats']
        display(Markdown(f"- **Creature Count:** {creature_stats['creature_count']} ({creature_stats['creature_count']/analysis['card_count']:.1%} of deck)"))
        if creature_stats['creature_count'] > 0:
            display(Markdown(f"- **Avg Power/Toughness:** {creature_stats['avg_power']:.1f}/{creature_stats['avg_toughness']:.1f}"))
            display(Markdown(f"- **Creature Mix:** Small: {creature_stats['creature_categories']['small']}, Medium: {creature_stats['creature_categories']['medium']}, Large: {creature_stats['creature_categories']['large']}"))
            display(Markdown(f"- **Creature Theme Alignment:** {creature_stats['theme_alignment_score']:.1f}"))
        
        if analysis['color_issues']:
            display(Markdown(f"- **Color Issues:** {len(analysis['color_issues'])} cards"))
    
    # Bottom performing decks
    display(Markdown(f"## Bottom {top_n} Least Coherent Decks"))
    for i, (deck_name, analysis) in enumerate(sorted_decks[-top_n:]):
        display(Markdown(f"### {len(sorted_decks)-top_n+i+1}. {deck_name}"))
        display(Markdown(f"- **Overall Score: {analysis['overall_coherence']:.1f}/100**"))
        display(Markdown(f"- **Expected Themes:** {', '.join(analysis['expected_themes'])}"))
        display(Markdown(f"- **Theme Match Score:** {analysis['theme_score']:.1f}"))
        display(Markdown(f"- **Color Coherence:** {analysis['color_coherence']:.1%}"))
        
        # Creature stats
        creature_stats = analysis['creature_stats']
        if creature_stats['creature_count'] > 0:
            display(Markdown(f"- **Avg Power/Toughness:** {creature_stats['avg_power']:.1f}/{creature_stats['avg_toughness']:.1f}"))
            display(Markdown(f"- **Creature Theme Alignment:** {creature_stats['theme_alignment_score']:.1f}"))
        
        if analysis['color_issues']:
            display(Markdown(f"- **Color Issues:** {analysis['color_issues'][:3]}"))

def analyze_specific_deck_enhanced(deck_name, cube_df, oracle_df, coherence_results):
    """Enhanced deck analysis including creature statistics"""
    
    if deck_name not in coherence_results:
        print(f"Deck '{deck_name}' not found!")
        return
    
    deck_data = coherence_results[deck_name]
    deck_cards = cube_df[cube_df['Tags'] == deck_name]
    creature_stats = deck_data['creature_stats']
    
    display(Markdown(f"# Enhanced Analysis: {deck_name}"))
    display(Markdown(f"**Overall Coherence Score: {deck_data['overall_coherence']:.1f}/100**"))
    
    # Theme analysis
    display(Markdown("## Theme Analysis"))
    display(Markdown(f"**Expected Themes:** {', '.join(deck_data['expected_themes'])}"))
    display(Markdown(f"**Theme Match Score:** {deck_data['theme_score']:.1f}"))
    
    # Creature Statistics
    display(Markdown("## Creature Statistics"))
    display(Markdown(f"**Total Creatures:** {creature_stats['creature_count']} ({creature_stats['creature_count']/deck_data['card_count']:.1%} of deck)"))
    
    if creature_stats['creature_count'] > 0:
        display(Markdown(f"**Average Power/Toughness:** {creature_stats['avg_power']:.1f}/{creature_stats['avg_toughness']:.1f}"))
        display(Markdown(f"**Total Power/Toughness:** {creature_stats['total_power']:.0f}/{creature_stats['total_toughness']:.0f}"))
        display(Markdown(f"**Creature Theme Alignment Score:** {creature_stats['theme_alignment_score']:.1f}"))
        
        # Creature categories
        display(Markdown("### Creature Categories:"))
        display(Markdown(f"- **Small (≤2 power):** {creature_stats['creature_categories']['small']} creatures"))
        display(Markdown(f"- **Medium (3-4 power):** {creature_stats['creature_categories']['medium']} creatures"))
        display(Markdown(f"- **Large (≥5 power):** {creature_stats['creature_categories']['large']} creatures"))
        display(Markdown(f"- **Evasive abilities:** {creature_stats['creature_categories']['evasive']} creatures"))
        display(Markdown(f"- **Utility abilities:** {creature_stats['creature_categories']['utility']} creatures"))
        
        # Power distribution
        if creature_stats['power_distribution']:
            power_dist = ", ".join([f"{k}: {v}" for k, v in sorted(creature_stats['power_distribution'].items())])
            display(Markdown(f"**Power Distribution:** {power_dist}"))
        
        # Show individual creatures
        if creature_stats['creature_details']:
            display(Markdown("### Creature Details:"))
            for creature in creature_stats['creature_details'][:10]:  # Show first 10
                categories_str = ", ".join(creature['categories']) if creature['categories'] else "basic"
                display(Markdown(f"- **{creature['name']}** ({creature['power']:.0f}/{creature['toughness']:.0f}) - {categories_str}"))
    
    # Color coherence
    display(Markdown("## Color Analysis"))
    display(Markdown(f"**Deck Colors:** {deck_data['deck_colors']}"))
    display(Markdown(f"**Color Coherence:** {deck_data['color_coherence']:.1%}"))
    
    if deck_data['color_issues']:
        display(Markdown("### Color Issues:"))
        for issue in deck_data['color_issues'][:5]:
            display(Markdown(f"- {issue}"))
    
    # Mana curve
    display(Markdown("## Mana Curve Analysis"))
    display(Markdown(f"**Mana Curve Score:** {deck_data['mana_curve_score']:.1%}"))
    
    curve_display = []
    for cmc in sorted(deck_data['mana_curve'].keys()):
        count = deck_data['mana_curve'][cmc]
        curve_display.append(f"CMC {cmc}: {count} cards")
    
    display(Markdown("**Curve Distribution:**"))
    for item in curve_display:
        display(Markdown(f"- {item}"))
    
    # Enhanced recommendations including creature analysis
    display(Markdown("## Improvement Recommendations"))
    
    recommendations = []
    
    if deck_data['theme_score'] < 2.0:
        recommendations.append("🎯 **Theme Focus**: Consider adding more cards that directly support the deck's themes")
    
    if deck_data['color_coherence'] < 0.9:
        recommendations.append("🎨 **Color Issues**: Some cards don't fit the color identity - consider replacements")
    
    if deck_data['mana_curve_score'] < 0.8:
        recommendations.append("⚡ **Mana Curve**: Consider adjusting the mana curve for better balance")
    
    # Creature-specific recommendations
    if creature_stats['creature_count'] > 0:
        if creature_stats['theme_alignment_score'] < 1.0:
            recommendations.append("👹 **Creature Synergy**: Creature stats don't align well with deck themes")
        
        expected_themes = deck_data['expected_themes']
        if 'Aggro' in expected_themes and creature_stats['creature_categories']['large'] > creature_stats['creature_categories']['small']:
            recommendations.append("⚡ **Aggro Focus**: Consider more small, efficient creatures for aggro strategy")
        
        if 'Big Creatures' in expected_themes and creature_stats['avg_power'] < 4:
            recommendations.append("💪 **Big Creatures**: Average creature power is low for a big creatures theme")
        
        if 'Flying' in expected_themes and creature_stats['creature_categories']['evasive'] < creature_stats['creature_count'] * 0.5:
            recommendations.append("🕊️ **Flying Theme**: Consider more creatures with flying or evasion")
    
    # Check for missing card types
    creature_count = creature_stats['creature_count']
    spell_count = len(deck_cards) - creature_count
    
    if creature_count < 4:
        recommendations.append("👹 **Creatures**: Consider adding more creatures for board presence")
    elif creature_count > 10:
        recommendations.append("📜 **Spells**: Consider adding more non-creature spells for versatility")
    
    if not recommendations:
        recommendations.append("✅ **Excellent Balance**: Deck shows great coherence across all metrics including creature synergy!")
    
    for rec in recommendations:
        display(Markdown(f"- {rec}"))

In [5]:
# cards = get_available_cards_for_deck("Green Big Creatures", cube_df, oracle_df, "G")
# [m.get('name') for m in cards]

In [6]:
from itertools import combinations
import re

def display_swap_recommendations(deck_name, swap_results, cube_df, oracle_df):
    """
    Display swap recommendations in a formatted way.
    """
    if not swap_results:
        display(Markdown(f"# No beneficial swaps found for {deck_name}"))
        return
    
    improvement = swap_results.get('improvement', 0)
    num_swaps = len(swap_results.get('remove', []))
    
    display(Markdown(f"# Optimal Swap Recommendations for {deck_name}"))
    display(Markdown(f"**Improvement: +{improvement:.1f} coherence points**"))
    display(Markdown(f"**Number of swaps: {num_swaps}**"))
    
    # Display cards to remove
    display(Markdown("## Cards to Remove:"))
    for removal in swap_results['remove']:
        card_name = removal['name']
        reasons = ", ".join(removal['reasons']) if removal['reasons'] else "Low synergy"
        theme_matches = ", ".join(removal['theme_matches']) if removal['theme_matches'] else "None"
        
        display(Markdown(f"### {card_name}"))
        display(Markdown(f"- **Score:** {removal['score']}/10"))
        display(Markdown(f"- **Theme Score:** {removal['theme_score']}/10"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Removal Reasons:** {reasons}"))
    
    # Display cards to add
    display(Markdown("## Cards to Add:"))
    for addition in swap_results['add']:
        card_name = addition['name']
        reasons = ", ".join(addition['reasons']) if addition['reasons'] else "Good synergy"
        theme_matches = ", ".join(addition['theme_matches']) if addition['theme_matches'] else "None"
        
        display(Markdown(f"### {card_name}"))
        display(Markdown(f"- **Score:** {addition['score']}/10"))
        display(Markdown(f"- **Theme Score:** {addition['theme_score']}/10"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Addition Reasons:** {reasons}"))

def optimize_deck_with_swaps(deck_name, cube_df, oracle_df, max_swaps=3):
    """
    Find the optimal swaps to improve deck coherence.
    """
    current_analysis = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)[deck_name]
    current_score = current_analysis['overall_coherence']
    deck_cards = cube_df[cube_df['Tags'].str.contains(deck_name, na=False)]['Name'].tolist()
    
    # Get available cards (not in any deck)
    used_cards = set()
    for deck in cube_df['Tags'].dropna().unique():
        deck_cards_for_this = cube_df[cube_df['Tags'].str.contains(deck, na=False)]['Name'].tolist()
        used_cards.update(deck_cards_for_this)
    
    available_cards = oracle_df[~oracle_df['name'].isin(used_cards)]['name'].tolist()
    
    best_swap = None
    best_improvement = 0
    
    # Try combinations of 1 to max_swaps cards
    for num_swaps in range(1, min(max_swaps + 1, len(deck_cards) + 1)):
        for cards_to_remove in combinations(deck_cards, num_swaps):
            # Analyze what makes these cards weak
            removal_analysis = []
            for card in cards_to_remove:
                card_data = oracle_df[oracle_df['name'] == card].iloc[0]
                analysis = analyze_single_card_for_deck(card_data, deck_name, oracle_df)
                removal_analysis.append({
                    'name': card,
                    'score': analysis['overall_score'],
                    'theme_score': analysis['theme_score'],
                    'theme_matches': analysis['theme_matches'],
                    'reasons': analysis['removal_reasons']
                })
            
            # Try combinations of available cards to add
            for cards_to_add in combinations(available_cards, num_swaps):
                # Create temporary modified cube
                temp_cube = cube_df.copy()
                
                # Remove cards
                for card in cards_to_remove:
                    temp_cube = temp_cube[temp_cube['Name'] != card]
                
                # Add cards
                for card in cards_to_add:
                    card_data = oracle_df[oracle_df['name'] == card]
                    if not card_data.empty:
                        new_row = card_data.iloc[0].copy()
                        # Map oracle columns to cube columns
                        cube_row = {
                            'Name': new_row['name'],
                            'Set': 'Mixed',
                            'Collector Number': None,
                            'Rarity': new_row.get('rarity', 'common'),
                            'Color Identity': new_row.get('color_identity', ''),
                            'Type': new_row.get('type_line', ''),
                            'Mana Cost': new_row.get('mana_cost', ''),
                            'CMC': new_row.get('mana_value', 0),
                            'Power': new_row.get('power', None),
                            'Toughness': new_row.get('toughness', None),
                            'Tags': deck_name
                        }
                        temp_cube = pd.concat([temp_cube, pd.DataFrame([cube_row])], ignore_index=True)
                
                # Calculate new score
                new_analysis = analyze_deck_theme_coherence_enhanced(temp_cube, oracle_df)
                if deck_name in new_analysis:
                    new_score = new_analysis[deck_name]['overall_coherence']
                    improvement = new_score - current_score
                    
                    if improvement > best_improvement:
                        # Analyze added cards
                        addition_analysis = []
                        for card in cards_to_add:
                            card_data = oracle_df[oracle_df['name'] == card].iloc[0]
                            analysis = analyze_single_card_for_deck(card_data, deck_name, oracle_df)
                            addition_analysis.append({
                                'name': card,
                                'score': analysis['overall_score'],
                                'theme_score': analysis['theme_score'],
                                'theme_matches': analysis['theme_matches'],
                                'reasons': analysis['addition_reasons']
                            })
                        
                        best_improvement = improvement
                        best_swap = {
                            'remove': removal_analysis,
                            'add': addition_analysis,
                            'improvement': improvement,
                            'new_score': new_score
                        }
    
    return best_swap

def analyze_single_card_for_deck(card_data, deck_name, oracle_df):
    """
    Analyze a single card's fit for a specific deck theme.
    """
    # Initialize scores
    theme_score = 0
    overall_score = 0
    theme_matches = []
    removal_reasons = []
    addition_reasons = []
    
    # Get card properties
    card_name = card_data['name']
    card_text = str(card_data.get('text', '')).lower()
    type_line = str(card_data.get('type_line', '')).lower()
    mana_cost = str(card_data.get('mana_cost', ''))
    
    # Extract mana value
    try:
        mana_value = float(card_data.get('mana_value', 0)) if pd.notna(card_data.get('mana_value')) else 0
    except (ValueError, TypeError):
        mana_value = 0
    
    # Get deck theme keywords
    deck_keywords = theme_keywords.get(deck_name, [])
    deck_colors = [color for color in ['W', 'U', 'B', 'R', 'G'] if any(color.lower() in keyword.lower() for keyword in deck_keywords)]
    
    # Theme keyword matching
    keyword_matches = 0
    for keyword in deck_keywords:
        keyword_lower = keyword.lower()
        if (keyword_lower in card_text or 
            keyword_lower in type_line or 
            keyword_lower in card_name.lower()):
            keyword_matches += 1
            theme_matches.append(f"{keyword}({keyword_matches})")
    
    theme_score = min(keyword_matches * 2, 10)
    
    # Color identity matching
    color_bonus = 0
    if deck_colors:
        card_colors = set(re.findall(r'[WUBRG]', mana_cost))
        if card_colors.issubset(set(deck_colors)):
            color_bonus = 2
            addition_reasons.append("Correct color identity")
        elif card_colors & set(deck_colors):
            color_bonus = 1
            addition_reasons.append("Partial color match")
        else:
            removal_reasons.append("Wrong colors")
    
    # Specific deck analysis
    if 'aggressive' in deck_name.lower() or 'aggro' in deck_name.lower():
        if mana_value <= 3:
            overall_score += 2
            addition_reasons.append("Low mana cost for aggro")
        elif mana_value >= 6:
            overall_score -= 2
            removal_reasons.append("Too expensive for aggro")
            
        if 'haste' in card_text:
            overall_score += 2
            addition_reasons.append("Has haste")
    
    elif 'control' in deck_name.lower():
        if mana_value >= 4:
            overall_score += 1
            addition_reasons.append("Good late-game card")
        if any(word in card_text for word in ['draw', 'counter', 'destroy', 'exile']):
            overall_score += 2
            addition_reasons.append("Control effect")
    
    elif 'big' in deck_name.lower() and 'creatures' in deck_name.lower():
        try:
            power = float(card_data.get('power', 0)) if pd.notna(card_data.get('power')) else 0
            toughness = float(card_data.get('toughness', 0)) if pd.notna(card_data.get('toughness')) else 0
            
            if 'creature' in type_line:
                if power >= 4 or toughness >= 4:
                    overall_score += 3
                    addition_reasons.append("Big creature stats")
                elif power <= 2 and toughness <= 2:
                    overall_score -= 2
                    removal_reasons.append("Small creature stats")
        except (ValueError, TypeError):
            pass
    
    # Calculate final scores
    overall_score = max(0, min(theme_score + color_bonus + overall_score, 10))
    
    return {
        'overall_score': overall_score,
        'theme_score': theme_score,
        'theme_matches': theme_matches,
        'removal_reasons': removal_reasons,
        'addition_reasons': addition_reasons
    }

def quick_deck_optimization(deck_name, cube_df, oracle_df, apply_changes=False):
    """
    Quick optimization function with clear output.
    """
    print(f"Current coherence: {analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)[deck_name]['overall_coherence']:.1f}")
    
    # Find optimal swaps
    swap_results = optimize_deck_with_swaps(deck_name, cube_df, oracle_df)
    
    # Display recommendations
    display_swap_recommendations(deck_name, swap_results, cube_df, oracle_df)
    
    # Optionally apply changes
    if apply_changes and swap_results:
        modified_cube = cube_df.copy()
        
        # Remove cards
        for removal in swap_results['remove']:
            modified_cube = modified_cube[modified_cube['Name'] != removal['name']]
        
        # Add cards
        for addition in swap_results['add']:
            card_data = oracle_df[oracle_df['name'] == addition['name']]
            if not card_data.empty:
                new_row = card_data.iloc[0].copy()
                # Map oracle columns to cube columns
                cube_row = {
                    'Name': new_row['name'],
                    'Set': 'Mixed',
                    'Collector Number': None,
                    'Rarity': new_row.get('rarity', 'common'),
                    'Color Identity': new_row.get('color_identity', ''),
                    'Type': new_row.get('type_line', ''),
                    'Mana Cost': new_row.get('mana_cost', ''),
                    'CMC': new_row.get('mana_value', 0),
                    'Power': new_row.get('power', None),
                    'Toughness': new_row.get('toughness', None),
                    'Tags': deck_name
                }
                modified_cube = pd.concat([modified_cube, pd.DataFrame([cube_row])], ignore_index=True)
        
        print(f"New coherence: {analyze_deck_theme_coherence_enhanced(modified_cube, oracle_df)[deck_name]['overall_coherence']:.1f}")
        return modified_cube, swap_results
    
    return cube_df, swap_results

In [7]:
def display_swap_recommendations(deck_name, swap_results, cube_df, oracle_df):
    """Display the swap recommendations in a clear, readable format"""
    
    if not swap_results:
        display(Markdown(f"# No Beneficial Swaps Found for {deck_name}"))
        display(Markdown("The current deck configuration appears to be optimal, or no suitable replacement cards were found."))
        return
    
    display(Markdown(f"# Optimal Swap Recommendations for {deck_name}"))
    
    improvement = swap_results['improvement']
    num_swaps = swap_results['num_swaps']
    
    display(Markdown(f"**Improvement: +{improvement:.1f} coherence points**"))
    display(Markdown(f"**Number of swaps: {num_swaps}**"))
    
    # Show cards to remove
    display(Markdown("## Cards to Remove:"))
    for i, removal in enumerate(swap_results['remove']):
        card_name = removal['name']
        reasons = ", ".join(removal['reasons']) if removal['reasons'] else "Low synergy"
        theme_matches = ", ".join(removal['theme_matches']) if removal['theme_matches'] else "None"
        
        display(Markdown(f"### {i+1}. **{card_name}**"))
        display(Markdown(f"- **Removal Score:** {removal['score']:.1f}/10"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Reasons:** {reasons}"))
    
    # Show cards to add
    display(Markdown("## Cards to Add:"))
    for i, addition in enumerate(swap_results['add']):
        card_name = addition['name']
        reasons = ", ".join(addition['reasons']) if addition['reasons'] else "Good synergy"
        theme_matches = ", ".join(addition['theme_matches']) if addition['theme_matches'] else "None"
        card_data = addition['card_data']
        
        # Get card details
        card_type = card_data.get('Type', 'Unknown')
        cmc = card_data.get('CMC', 'Unknown')
        power = card_data.get('Power', '')
        toughness = card_data.get('Toughness', '')
        
        power_toughness = ""
        if not pd.isna(power) and not pd.isna(toughness):
            power_toughness = f" ({power}/{toughness})"
        
        display(Markdown(f"### {i+1}. **{card_name}**{power_toughness}"))
        display(Markdown(f"- **Type:** {card_type}"))
        display(Markdown(f"- **CMC:** {cmc}"))
        display(Markdown(f"- **Addition Score:** {addition['score']:.1f}"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Reasons:** {reasons}"))
        
        # Show oracle text (truncated)
        oracle_text = str(card_data.get('Oracle Text', '')).strip()
        if oracle_text and oracle_text != 'nan':
            truncated_text = oracle_text[:150] + "..." if len(oracle_text) > 150 else oracle_text
            display(Markdown(f"- **Oracle Text:** {truncated_text}"))
    
    # Show summary
    display(Markdown("## Summary"))
    display(Markdown(f"This {num_swaps}-card swap will improve the deck's coherence score by **{improvement:.1f} points**."))
    display(Markdown("The recommended changes focus on:"))
    
    # Analyze the themes of added cards
    added_themes = set()
    for addition in swap_results['add']:
        added_themes.update(addition['theme_matches'])
    
    if added_themes:
        display(Markdown(f"- Strengthening synergies: {', '.join(list(added_themes)[:5])}"))
    
    # Count creature additions/removals
    creatures_removed = sum(1 for r in swap_results['remove'] 
                           if 'creature' in str(cube_df[cube_df['Name'] == r['name']]['Type'].iloc[0]).lower())
    creatures_added = sum(1 for a in swap_results['add'] 
                         if 'creature' in str(a['card_data'].get('Type', '')).lower())
    
    if creatures_added != creatures_removed:
        diff = creatures_added - creatures_removed
        if diff > 0:
            display(Markdown(f"- Adding {diff} more creature(s) for better board presence"))
        else:
            display(Markdown(f"- Removing {abs(diff)} creature(s) for more spell-based strategy"))

def apply_swap_recommendations(deck_name, swap_results, cube_df):
    """Apply the recommended swaps to the cube dataframe"""
    
    if not swap_results:
        print("No swaps to apply!")
        return cube_df
    
    modified_cube = cube_df.copy()
    
    print(f"Applying {swap_results['num_swaps']}-card swap to {deck_name}...")
    
    # Remove cards from deck
    for removal in swap_results['remove']:
        card_name = removal['name']
        mask = (modified_cube['Name'] == card_name) & (modified_cube['Tags'] == deck_name)
        if mask.any():
            modified_cube.loc[mask, 'Tags'] = 'REMOVED'
            print(f"  Removed: {card_name}")
    
    # Add cards to deck
    for addition in swap_results['add']:
        card_name = addition['name']
        card_data = addition['card_data']
        
        # Check if card already exists in cube
        existing_card = modified_cube[modified_cube['Name'] == card_name]
        if not existing_card.empty:
            # Update existing card's assignment
            modified_cube.loc[modified_cube['Name'] == card_name, 'Tags'] = deck_name
            print(f"  Reassigned: {card_name}")
        else:
            # Add new card to cube
            new_row = {
                'Name': card_name,
                'Tags': deck_name,
                'Type': card_data.get('Type', ''),
                'CMC': card_data.get('CMC', 0)
            }
            new_df = pd.DataFrame([new_row])
            modified_cube = pd.concat([modified_cube, new_df], ignore_index=True)
            print(f"  Added: {card_name}")
    
    print(f"Swap complete! Expected improvement: +{swap_results['improvement']:.1f} coherence points")
    modified_cube.drop(modified_cube[modified_cube.Tags == 'REMOVED'].index, inplace=True)  # Clean up removed cards

    return modified_cube

def quick_deck_optimization(deck_name, cube_df, oracle_df, apply_changes=False):
    """Quick function to optimize a deck and optionally apply changes"""
    
    print(f"🎯 Optimizing {deck_name}...")
    
    # Get swap recommendations
    swap_results = optimize_deck_with_swaps(deck_name, cube_df, oracle_df)
    
    # Display recommendations
    display_swap_recommendations(deck_name, swap_results, cube_df, oracle_df)
    
    # Optionally apply changes
    if apply_changes and swap_results:
        modified_cube = apply_swap_recommendations(deck_name, swap_results, cube_df)
        return modified_cube, swap_results
    
    return cube_df, swap_results

In [8]:
def show_available_cards_for_deck(deck_name, cube_df, oracle_df, top_n=20):
    """Show the top available cards that could be added to a deck"""
    
    # Get deck colors and themes
    deck_colors = get_deck_colour(deck_name)
    expected_themes = extract_theme_from_deck_name(deck_name)
    
    print(f"Top available cards for {deck_name} (Colors: {deck_colors}, Themes: {expected_themes})")
    print("="*80)
    
    # Get available cards
    available_cards = get_available_cards_for_deck(deck_name, cube_df, oracle_df, deck_colors)
    
    if not available_cards:
        print("No available cards found!")
        return
    
    # Score available cards
    addition_candidates = score_cards_for_addition(available_cards, expected_themes, deck_colors, oracle_df)
    
    # Show top candidates
    display(Markdown(f"# Top {top_n} Available Cards for {deck_name}"))
    
    for i, card in enumerate(addition_candidates[:top_n]):
        card_name = card['name']
        card_data = card['card_data']
        score = card['score']
        reasons = ", ".join(card['reasons']) if card['reasons'] else "Basic utility"
        theme_matches = ", ".join(card['theme_matches']) if card['theme_matches'] else "None"
        
        # Get card details
        card_type = card_data.get('Type', 'Unknown')
        cmc = card_data.get('CMC', 'Unknown')
        power = card_data.get('Power', '')
        toughness = card_data.get('Toughness', '')
        
        power_toughness = ""
        if not pd.isna(power) and not pd.isna(toughness):
            power_toughness = f" ({power}/{toughness})"
        
        display(Markdown(f"## {i+1}. **{card_name}**{power_toughness} (Score: {score:.1f})"))
        display(Markdown(f"- **Type:** {card_type} | **CMC:** {cmc}"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Reasons:** {reasons}"))
        
        # Show oracle text (truncated)
        oracle_text = str(card_data.get('Oracle Text', '')).strip()
        if oracle_text and oracle_text != 'nan':
            truncated_text = oracle_text[:200] + "..." if len(oracle_text) > 200 else oracle_text
            display(Markdown(f"- **Oracle Text:** *{truncated_text}*"))
        
        display(Markdown("---"))

def show_removal_candidates_for_deck(deck_name, cube_df, oracle_df, top_n=10):
    """Show the cards most suitable for removal from a deck"""
    
    # Get current deck
    current_deck = cube_df[cube_df['Tags'] == deck_name]
    if current_deck.empty:
        print(f"Deck '{deck_name}' not found!")
        return
    
    # Get themes
    expected_themes = extract_theme_from_deck_name(deck_name)
    
    # Score cards for removal
    removal_candidates = score_cards_for_removal(current_deck, expected_themes, oracle_df)
    
    display(Markdown(f"# Top {top_n} Removal Candidates from {deck_name}"))
    
    for i, card in enumerate(removal_candidates[:top_n]):
        card_name = card['name']
        score = card['score']
        reasons = ", ".join(card['reasons']) if card['reasons'] else "Low synergy"
        theme_matches = ", ".join(card['theme_matches']) if card['theme_matches'] else "None"
        
        # Get card details from cube
        card_row = current_deck[current_deck['Name'] == card_name].iloc[0]
        card_type = card_row.get('Type', 'Unknown')
        cmc = card_row.get('CMC', 'Unknown')
        
        display(Markdown(f"## {i+1}. **{card_name}** (Removal Score: {score:.1f}/10)"))
        display(Markdown(f"- **Type:** {card_type} | **CMC:** {cmc}"))
        display(Markdown(f"- **Theme Matches:** {theme_matches}"))
        display(Markdown(f"- **Reasons for Removal:** {reasons}"))
        display(Markdown("---"))

In [9]:
# validation_results = validate_jumpstart_cube(cube_df, oracle_df)
# display_validate_results(validation_results)

In [10]:
# coherence_results = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
# display_coherence_analysis_enhanced(coherence_results)

In [ ]:
# analyze_specific_deck_enhanced("Green Big Creatures", cube_df, oracle_df,analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))


analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)["Green Big Creatures"]['overall_coherence']

NameError: name 'analyze_deck_theme_coherence_enhanced' is not defined

In [ ]:


display_validate_results(validate_jumpstart_cube(cube_df, oracle_df))

### Cube is valid! 🎉

#### Deck Summaries:

**Azorius Evasion/Flying**: {'total_cards': 13, 'lands': 2, 'non_lands': 11, 'color_issues': [], 'cards': ['Archaeomancer', 'Behold the Multiverse', 'Ephemerate', 'Lyev Skyknight', 'Lórien Revealed', 'Momentary Blink', 'Oblivion Ring', 'Peregrine Drake', 'Silver Drake', 'Azorius Chancery', 'Skybridge Towers', "Judge's Familiar", 'Bubble Snare']}

**Black Aggro**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Carrier Thrall', 'Carrion Feeder', 'Dauthi Slayer', 'Gixian Infiltrator', 'Loathsome Curator', 'Scurrilous Sentry', 'Tithing Blade', 'Tortured Existence', 'Troll of Khazad-dûm', 'Vampire Lacerator', 'Vampire Sovereign', 'Wither and Bloom', 'Grasping Scoundrel']}

**Black Control**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Accursed Marauder', 'Doom Blade', 'Falkenrath Noble', 'Gift of Fangs', 'Gray Merchant of Asphodel', 'Infestation Sage', 'Phyrexian Rager', 'Unearth', 'Unwilling Ingredient', 'Vendetta', 'Voracious Vermin', 'Plagued Rusalka', 'Guildsworn Prowler']}

...

**Simic Control**: {'total_cards': 13, 'lands': 3, 'non_lands': 10, 'color_issues': [], 'cards': ['Bramble Wurm', 'Coiling Oracle', 'Experiment One', 'Frost Trickster', 'Growth Spiral', 'Monstrous Emergence', 'Quandrix Pledgemage', 'Sprout Swarm', 'Quandrix Campus', 'Simic Growth Chamber', 'Tangled Islet', 'Snapping Voidcraw', 'Gitaxian Probe']}

**White Aggro**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Aerie Auxiliary', 'Ainok Bond-Kin', 'Cartouche of Solidarity', 'Faerie Guidemother', "Gideon's Lawkeeper", 'Inspiring Paladin', 'Mandibular Kite', 'Palace Sentinels', 'Pegasus Guardian', "Raffine's Informant", 'Sandsteppe Outcast', 'Stormfront Pegasus', 'Boros Elite']}

**White Control**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Dauntless Unity', 'Inspiring Overseer', 'Journey to Nowhere', 'Petrify', 'Rhox Veteran', 'Search Party Captain', 'Suture Priest', 'Thraben Charm', 'Thraben Inspector', 'You Hear Something on Watch', 'Cathar Commando', 'Alabaster Host Intercessor', 'Borrowed Grace']}

**White Equipment**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Custodi Squire', 'Deftblade Elite', 'Doomed Traveler', "Heliod's Pilgrim", 'Bonesplitter', "Miner's Guidewing", 'Phantom Nomad', 'Planar Disruption', 'Vulshok Morningstar', 'Seeker of the Way', 'Glimmerlight', 'Ancestral Blade', 'Greatsword of Tyr']}

**White Tokens**: {'total_cards': 13, 'lands': 0, 'non_lands': 13, 'color_issues': [], 'cards': ['Ardenvale Tactician', 'Coalition Honor Guard', 'Combat Professor', 'Dog Umbra', 'Eagles of the North', 'Holy Cow', 'Kor Skyfisher', 'Savannah Lions', 'Settle Beyond Reality', 'Sunlance', 'Syndic of Tithes', 'Battle Screech', 'Raise the Alarm']}

In [ ]:
deck = "Green Big Creatures"
print(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)[deck])

{'expected_themes': ['Big Creatures'], 'theme_score': 0.5384615384615384, 'theme_matches': [{'card': 'Colossal Dreadmask', 'score': 2, 'themes': ['Big Creatures(2)']}, {'card': 'Generous Ent', 'score': 1, 'themes': ['Big Creatures(5+ power/toughness)']}, {'card': 'Hazard of the Dunes', 'score': 2, 'themes': ['Big Creatures(2)']}, {'card': 'Jade Guardian', 'score': 0, 'themes': []}, {'card': 'Jewel Thief', 'score': 1, 'themes': ['Big Creatures(1)']}, {'card': 'Krosan Tusker', 'score': 1, 'themes': ['Big Creatures(5+ power/toughness)']}, {'card': 'Lead the Stampede', 'score': 0, 'themes': []}, {'card': 'Malevolent Rumble', 'score': 0, 'themes': []}, {'card': "Sarulf's Packmate", 'score': 0, 'themes': []}, {'card': 'Urban Daggertooth', 'score': 0, 'themes': []}, {'card': 'Utopia Sprawl', 'score': 0, 'themes': []}, {'card': 'Wary Thespian', 'score': 0, 'themes': []}, {'card': 'Basking Broodscale', 'score': 0, 'themes': []}], 'color_coherence': 1.0, 'color_issues': [], 'mana_curve_score': 0

In [ ]:

print(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)[deck]['overall_coherence'])

modified_cube, swap_results = quick_deck_optimization(deck, cube_df, oracle_df)

# print(analyze_deck_theme_coherence_enhanced(modified_cube, oracle_df)[deck]['overall_coherence'])

# Apply the changes if you like them
if swap_results:
    cube_df = apply_swap_recommendations(deck, swap_results, cube_df)

print(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)[deck]['overall_coherence'])


43.007384615384616
🎯 Optimizing Green Big Creatures...


KeyboardInterrupt: 

# Generated Below

In [ ]:
# Example: Export the current cube to CSV
# This will create a file with timestamp in the name
filename = quick_export_cube(cube_df, oracle_df)

# Validate the export
validate_export(filename, cube_df)